This is a model of the statistical mechanics of a growing crystal. Cubes are added at random to the outside of the crystal, with a probability that depends on the number of neighbours. By playing with these probabilities one can find "crystals" that are asymptotically cubic or spherical (I think). 

There's an interesting question of how one could modify the model to get other kinds of facets, e.g. in the (111) and related directions.

In [18]:
#NAME: Crystal Growth
#DESCRIPTION: Statistical mechanics model of the growth of a crystal.

from vpython import *
import numpy as np

In [19]:
cube_size = 10
x_offset, y_offset, z_offset = 0, 0, 0

`surface` keep track of coordinates of outer surface elements (cubes that could be added next). This is the initial outer surface

In [20]:
surface = [np.array((1,0,0), dtype = np.int), 
           np.array((-1,0,0), dtype = np.int), 
           np.array((0,1,0), dtype = np.int), 
           np.array((0,-1,0), dtype = np.int), 
           np.array((0,0,1), dtype = np.int), 
           np.array((0,0,-1), dtype = np.int)]

The 3D array `sites` containing the number of neighbours for a given occupied site, while `site_list` lists the occupied sites. 

In [21]:
L = 1000
sites = np.zeros((L, L, L), dtype = np.int)
site_list = [np.array((0,0,0), dtype = np.int)]    

Now we initialize the neighbours

In [22]:
sites[0,0,0] = 6
sites[1,0,0] = 1
sites[-1,0,0] = 1
sites[0,1,0] = 1
sites[0,-1,0] = 1
sites[0,0,1] = 1
sites[0,0,-1] = 1

The acceptance probabilities of adding box with given neighbours

In [23]:
accept_probs = [0.01, 0.01, 1, 1, 1, 1]

The first and second neighbour vectors

In [24]:
first_neighbours = [(1,0,0), (-1,0,0), (0,1,0), (0,-1,0) ,(0,0,1), (0,0,-1)]

second_neighbours = []
for a in (1,-1):
    for b in (1,-1):
        for c in ((a,b,0), (a,0,b), (0,a,b)):
            second_neighbours.append(c)

Note that if we don't just have corner growth, it is possible to have enclosed regions 
on the outer surface with 6 neighbours enclosing

Here is the main loop. We choose an outer surface element at random, and add it depending on the acceptance probability.

When we add a new surface element we increment the number of neighbours for all neighbouring cubes (up to max of six).

Those incremented for the first time are added to the outer surface element list. Sites with added cubes are deleted from the outer surface element list.

In [25]:
fmin = 0 # This is the number of faces you need to be added to surface

for step in range(100000):
    surface_index = np.random.randint(len(surface))
    surface_box = surface[surface_index]  # Choose point on the outer surface at random
    occupied_faces = sites[tuple(surface_box)] # The number of occupied faces
    
    # Check that there are occupied second neighbours. If there are not, don't fill
    if step > 1000 & any([sites[tuple(surface_box + vec)] for vec in second_neighbours]):
    
        if np.random.rand() < accept_probs[occupied_faces - 1]:         # We are going to add the cube

            site_list.append(surface_box)
            # box(pos=vector(x_offset + surface_box[0] * cube_size, y_offset + surface_box[1] * cube_size,z_offset + surface_box[2] * cube_size),size = vector(cube_size,cube_size,cube_size)) # Draw the box

            # remove site from outer surface
            del(surface[surface_index])

            #increment surrounding sites and add to surface if new

            if step == 1000:
                fmin = 2 # After some time, we only add sites to the surface if they have fmin + 1 neighbours
            
            for neighbour in first_neighbours:
                
                if sites[tuple(surface_box + neighbour)] == fmin:
                    surface.append(surface_box + neighbour)
                sites[tuple(surface_box + neighbour)] += 1 

            
        
            



Display the completed crystal, excluding internal sites (with 6 neighbours).

In [26]:
scene = canvas()
scene.forward = vector(-1, -1, -1)

for site_index in site_list:
    if sites[tuple(site_index)] < 6:
        box(pos=vector(x_offset + site_index[0] * cube_size, y_offset + site_index[1] * cube_size,z_offset + site_index[2] * cube_size),size = vector(cube_size,cube_size,cube_size), color = color.white)

<IPython.core.display.Javascript object>